In this assignment, I used the BeautifulSoup python library with pandas to retrieve the wikipeia page  
<a>https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M</a>

# Part 1

In [1]:
#!conda install lxml
#!pip install beautifulsoup4 html5lib

In [2]:
import pandas as pd

In [3]:
tables = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', header = 0)

In [4]:
len(tables)

3

<div><b>Note</b>
<ul>
<li>pandas has a built-in function, read_html, which uses libraries like lxml and Beauti‐ful Soup to automatically parse tables out of HTML files as DataFrame objects.</li>
<li>By default it searches for and attempts to parse all tabular data contained within "table" tags. The result is a list of DataFrame objects.</li>
<li>The table was retrieved into "tables" and the first part contains the required data.</li>
</ul>
    </div>

In [5]:
df_toronto = tables[0]
df_toronto.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [6]:
#Change "Postcode" to "PostalCode" and "Neighbourhood" to "Neighborhood"
df_toronto.columns = ["PostalCode", "Borough", "Neighborhood"]

Dropping cells with a Borough that is <strong>Not assigned</strong>

In [7]:
df_toronto.groupby("Borough").count()

,PostalCode,Neighborhood
Borough,,
Central Toronto,17,17
Downtown Toronto,37,37
East Toronto,7,7
East York,6,6
Etobicoke,45,45
Mississauga,1,1
North York,38,38
Not assigned,77,77
Queen's Park,1,1


In [8]:
#Remove rows with "Not assiged" in rows
df_toronto = df_toronto[df_toronto.Borough != "Not assigned"]

In [9]:
#Reset index on rows
df_toronto = df_toronto.reset_index(drop=True)

df_toronto.shape

(212, 3)

Convert Neighbourhood that have "Not Assigned to have same name as Borough 

In [10]:
df_toronto[df_toronto.Neighborhood == "Not assigned"]

,PostalCode,Borough,Neighborhood
6,M7A,Queen's Park,Not assigned


In [11]:
df_toronto.replace({"Neighborhood": {"Not assigned":"Queen's Park"}}, inplace=True)

Clean cells that have more than one neighborhood existing in one postal code area.

In [12]:
## Combine the Neighborhood of same Postal code. 
df_toronto_comb = df_toronto.groupby(['PostalCode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
df_toronto_comb.head(10)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [13]:
df_toronto_comb.shape

(103, 3)

# Part 2

Let's get the latitude and the longitude coordinates of each neighborhood.

In [16]:
#!wget -q -O 'Geospatial_Coordinates.csv' https://cocl.us/Geospatial_data
print('GeoData downloaded!')

GeoData downloaded!


In [17]:
geodata_df = pd.read_csv("Geospatial_Coordinates.csv")
geodata_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [18]:
df_toronto_loc = pd.merge(df_toronto_comb, geodata_df, \
                          left_on='PostalCode', \
                          right_on='Postal Code', \
                          how='left').drop('Postal Code', axis=1)
df_toronto_loc

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [19]:
df_toronto_loc.shape

(103, 5)

# Part 3

In [21]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes  
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  53.33 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  27.94 MB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00  31.55 MB/s
folium-0.5.0-p 100% |################################| Time: 0:00:00  42.73 MB/s
Libraries imported.


In [23]:
address = 'Toronto, ON'

geolocator = Nominatim()

try:
    location = geolocator.geocode(address)
except:
    print("Service not available")
    location = None
if location is not None:
    latitude = location.latitude   
    longitude = location.longitude
    
else:
    latitude = 43.653963
    longitude = -79.387207
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [24]:
# Let's create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto_loc['Latitude'], df_toronto_loc['Longitude'], df_toronto_loc['Borough'], df_toronto_loc['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Let's explore the Queen's Park borough

In [25]:
## Define Foursquare Credentials and Version
CLIENT_ID = 'D3CF4ZFXEZVJDJ4CX0IOHO0YHGN0P1KD25ZVUH2QUINDYJQS' # your Foursquare ID
CLIENT_SECRET = 'Z0LIABRUZUCM3SXQKQGGOVYHBJXNYNWVLPCPOCL5FAHQGJM0' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: D3CF4ZFXEZVJDJ4CX0IOHO0YHGN0P1KD25ZVUH2QUINDYJQS
CLIENT_SECRET:Z0LIABRUZUCM3SXQKQGGOVYHBJXNYNWVLPCPOCL5FAHQGJM0


In [26]:
qp_data = df_toronto_loc[df_toronto_loc["Neighborhood"] == "Queen's Park" ]
qp_data

,PostalCode,Borough,Neighborhood,Latitude,Longitude
85,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


In [27]:
latitude = qp_data["Latitude"].values[0]
longitude = qp_data["Longitude"].values[0]
print('The geograpical coordinate of Queen\'s Park are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Queen's Park are 43.6623015, -79.3894938.


In [28]:
LIMIT = 150
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=D3CF4ZFXEZVJDJ4CX0IOHO0YHGN0P1KD25ZVUH2QUINDYJQS&client_secret=Z0LIABRUZUCM3SXQKQGGOVYHBJXNYNWVLPCPOCL5FAHQGJM0&ll=43.6623015,-79.3894938&v=20180605&radius=500&limit=150'

In [29]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c37db19dd57975fd5523c63'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4b9d206bf964a520e69136e3-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/park_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d163941735',
         'name': 'Park',
         'pluralName': 'Parks',
         'primary': True,
         'shortName': 'Park'}],
       'id': '4b9d206bf964a520e69136e3',
       'location': {'address': 'University Ave.',
        'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'crossStreet': 'at Wellesley Ave.',
        'distance': 283,
        'formattedAddress': ['University Ave. (at Wellesley Ave.)',
         'Toronto ON M5R 2E8',
         'Canada'],
        'labeledLatLngs': [{'l

In [30]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [31]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Queen's Park,Park,43.663946,-79.392180
1,Mercatto,Italian Restaurant,43.660391,-79.387664
2,Nando's Flame-Grilled Chicken,Portuguese Restaurant,43.661617,-79.386095
3,Coffee Island,Coffee Shop,43.664271,-79.386972
4,Sushi Box,Sushi Restaurant,43.662960,-79.386580


In [32]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

44 venues were returned by Foursquare.


# Let's visualize all neigbourhoods in North York Borough

In [69]:
ny_df = df_toronto_loc[df_toronto_loc["Borough"] == "North York"].reset_index(drop=True)
ny_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M2H,North York,Hillcrest Village,43.803762,-79.363452
1,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556
2,M2K,North York,Bayview Village,43.786947,-79.385975
3,M2L,North York,"Silver Hills, York Mills",43.757490,-79.374714
4,M2M,North York,"Newtonbrook, Willowdale",43.789053,-79.408493


In [34]:
## create a function to repeat the same process to all the neighborhoods in Toronto

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [59]:
# find the venues in all Toronto Neighbohoods
northyork_venues = getNearbyVenues(names=ny_df['Neighborhood'],
                                   latitudes=ny_df['Latitude'],
                                   longitudes=ny_df['Longitude']
                                  )

Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Bedford Park, Lawrence Manor East
Lawrence Heights, Lawrence Manor
Glencairn
Maple Leaf Park, North Park, Upwood Park
Humber Summit
Emery, Humberlea


In [60]:
northyork_venues.shape

(231, 7)

In [61]:
northyork_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Hillcrest Village,43.803762,-79.363452,Eagle's Nest Golf Club,43.805455,-79.364186,Golf Course
1,Hillcrest Village,43.803762,-79.363452,New York Fries,43.803664,-79.363905,Fast Food Restaurant
2,Hillcrest Village,43.803762,-79.363452,AY Jackson Pool,43.804515,-79.366138,Pool
3,Hillcrest Village,43.803762,-79.363452,Villa Madina,43.801685,-79.363938,Mediterranean Restaurant
4,Hillcrest Village,43.803762,-79.363452,Duncan Creek Park,43.805539,-79.360695,Dog Run


Let's check how many venues were returned for each neighborhood

In [62]:
## check how many venues were returned for each neighborhood
northyork_venues.groupby('Neighborhood').count().sort_values(by="Venue", ascending = False)

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Fairview, Henry Farm, Oriole",63,63,63,63,63,63
Willowdale South,35,35,35,35,35,35
"Bedford Park, Lawrence Manor East",23,23,23,23,23,23
"Flemingdon Park, Don Mills South",20,20,20,20,20,20
"Bathurst Manor, Downsview North, Wilson Heights",18,18,18,18,18,18
"Lawrence Heights, Lawrence Manor",12,12,12,12,12,12
"Northwood Park, York University",6,6,6,6,6,6
Hillcrest Village,5,5,5,5,5,5
Don Mills North,5,5,5,5,5,5


In [65]:
len(northyork_venues["Neighborhood"].unique())

23

23 neighborhoods were collected with venues. A neighborhood didn't have venues around it.

In [70]:
neighborhood_list = northyork_venues["Neighborhood"].unique()

#Drop row if neighborhood is not found in list
ny_df = ny_df[ny_df.Neighborhood.isin(neighborhood_list)]

In [71]:
ny_df.shape

(23, 5)

In [72]:
## find out how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(northyork_venues['Venue Category'].unique())))

There are 104 uniques categories.


In [73]:
## Analyze Each Neighborhood

# one hot encoding
northyork_onehot = pd.get_dummies(northyork_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
northyork_onehot['Neighborhood'] = northyork_venues['Neighborhood'] 

# move neighborhood column to the first column
column_list = northyork_onehot.columns.tolist()
column_number = int(column_list.index('Neighborhood'))
column_list = [column_list[column_number]] + column_list[:column_number] + column_list[column_number+1:]
northyork_onehot = northyork_onehot[column_list]

northyork_onehot.head()

,Neighborhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Store,Bike Shop,Boutique,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Café,Candy Store,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Electronics Store,Event Space,Fast Food Restaurant,Food & Drink Shop,Food Court,Food Truck,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,General Entertainment,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Hockey Arena,Hotel,Indian Restaurant,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Liquor Store,Lounge,Luggage Store,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Movie Theater,Moving Target,Park,Pet Store,Pharmacy,Pizza Place,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Restaurant,Salon / Barbershop,Sandwich Place,Shoe Store,Shopping Mall,Smoothie Shop,Sporting Goods Shop,Steakhouse,Supplement Shop,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint,Women's Store
0,Hillcrest Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Hillcrest Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Hillcrest Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Hillcrest Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Hillcrest Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [74]:
print(northyork_onehot.shape)

(231, 105)


In [75]:
## group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

northyork_grouped = northyork_onehot.groupby('Neighborhood').mean().reset_index()
northyork_grouped

,Neighborhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Store,Bike Shop,Boutique,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Café,Candy Store,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Electronics Store,Event Space,Fast Food Restaurant,Food & Drink Shop,Food Court,Food Truck,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,General Entertainment,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Hockey Arena,Hotel,Indian Restaurant,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Liquor Store,Lounge,Luggage Store,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Movie Theater,Moving Target,Park,Pet Store,Pharmacy,Pizza Place,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Restaurant,Salon / Barbershop,Sandwich Place,Shoe Store,Shopping Mall,Smoothie Shop,Sporting Goods Shop,Steakhouse,Supplement Shop,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint,Women's Store
0,"Bathurst Manor, Downsview North, Wilson Heights",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556,0.000000,0.000000,0.00,0.0,0.00,0.000000,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.00,0.000000,0.055556,0.000000,0.000000,0.00,0.055556,0.000000,0.0,0.000000,0.000000,0.055556,0.000000,0.000000,0.000000,0.055556,0.055556,0.00,0.00,0.055556,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.055556,0.000000,0.000000,0.00,0.000000,0.000000,0.055556,0.055556,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.055556,0.000000,0.055556,0.000000,0.055556,0.000000,0.000000,0.000000,0.000000,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556,0.000000,0.000000,0.000000
1,Bayview Village,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.00,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Bedford Park, Lawrence Manor East",0.000000,0.000000,0.043478,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.043478,0.043478,0.000000,0.000000,0.000000,0.000000,0.086957,0.043478,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.086957,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.0,0.043478,0.043478,0.0,0.0,0.043478,0.00,0.000000,0.043478,0.000000,0.00,0.086957,0.000000,0.000000,0.043478,0.043478,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.043478,0.043478,0.000000,0.0,0.00,0.043478,0.000000

In [76]:
northyork_grouped.shape

(23, 105)

In [77]:
## print each neighborhood along with the top 5 most common venues

num_top_venues = 5

for hood in northyork_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = northyork_grouped[northyork_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bathurst Manor, Downsview North, Wilson Heights----
                 venue  freq
0          Coffee Shop  0.11
1  Fried Chicken Joint  0.06
2          Bridal Shop  0.06
3        Deli / Bodega  0.06
4             Pharmacy  0.06


----Bayview Village----
                 venue  freq
0   Chinese Restaurant  0.25
1                 Café  0.25
2                 Bank  0.25
3  Japanese Restaurant  0.25
4    Accessories Store  0.00


----Bedford Park, Lawrence Manor East----
                  venue  freq
0  Fast Food Restaurant  0.09
1    Italian Restaurant  0.09
2           Coffee Shop  0.09
3      Sushi Restaurant  0.04
4                  Café  0.04


----CFB Toronto, Downsview East----
           venue  freq
0        Airport  0.33
1           Park  0.33
2       Bus Stop  0.33
3  Luggage Store  0.00
4      Pet Store  0.00


----Don Mills North----
                  venue  freq
0  Gym / Fitness Center   0.2
1  Caribbean Restaurant   0.2
2                  Café   0.2
3   Japanese Restaurant 

In [78]:
## put that into a pandas dataframe
## First, write a function to sort the venues in descending order.

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [79]:
## create the new dataframe and display the top 10 venues for each neighborhood.

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = northyork_grouped['Neighborhood']

for ind in np.arange(northyork_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(northyork_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor, Downsview North, Wilson Heights",Coffee Shop,Gift Shop,Bank,Pharmacy,Pizza Place,Deli / Bodega,Bridal Shop,Restaurant,Diner,Shopping Mall
1,Bayview Village,Chinese Restaurant,Café,Bank,Japanese Restaurant,Dog Run,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega,Department Store
2,"Bedford Park, Lawrence Manor East",Fast Food Restaurant,Italian Restaurant,Coffee Shop,Grocery Store,Sandwich Place,Greek Restaurant,Hardware Store,Indian Restaurant,Juice Bar,Liquor Store
3,"CFB Toronto, Downsview East",Park,Airport,Bus Stop,Dog Run,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega,Department Store
4,Don Mills North,Gym / Fitness Center,Caribbean Restaurant,Café,Basketball Court,Japanese Restaurant,Women's Store,Dog Run,Construction & Landscaping,Cosmetics Shop,Deli / Bodega
5,Downsview Central,Business Service,Food Truck,Baseball Field,Women's Store,Dog Run,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega,Department Store
6,Downsview Northwest,Grocery Store,Liquor Store,Athletics & Sports,Discount Store,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop
7,Downsview West,Grocery Store,Moving Target,Hotel,Bank,Dog Run,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega,Department Store
8,"Emery, Humberlea",Baseball Field,Women's Store,Electronics Store,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant
9,"Fairview, Henry Farm, Oriole",Clothing Store,Fast Food Restaurant,Coffee Shop,Tea Room,Men's Store,Restaurant,Bakery,Women's Store,Toy / Game Store,Deli / Bodega


Let's cluster neighbourhoods with same characteristics

In [90]:
## Cluster Neighborhoods
## Run k-means to cluster the neighborhood into 3 clusters.

# set number of clusters
kclusters = 3

northyork_grouped_clustering = northyork_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(northyork_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 2, 0, 0, 0, 0, 1, 0], dtype=int32)

In [92]:
# add clustering labels
ny_merged = ny_df.copy()
ny_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
ny_merged = ny_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

ny_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M2H,North York,Hillcrest Village,43.803762,-79.363452,0,Golf Course,Fast Food Restaurant,Pool,Mediterranean Restaurant,Dog Run,Frozen Yogurt Shop,Fried Chicken Joint,Coffee Shop,Comfort Food Restaurant,Gift Shop
1,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,0,Clothing Store,Fast Food Restaurant,Coffee Shop,Tea Room,Men's Store,Restaurant,Bakery,Women's Store,Toy / Game Store,Deli / Bodega
2,M2K,North York,Bayview Village,43.786947,-79.385975,0,Chinese Restaurant,Café,Bank,Japanese Restaurant,Dog Run,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega,Department Store
3,M2L,North York,"Silver Hills, York Mills",43.757490,-79.374714,2,Park,Women's Store,Greek Restaurant,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop
5,M2N,North York,Willowdale South,43.770120,-79.408493,0,Restaurant,Ramen Restaurant,Fast Food Restaurant,Sandwich Place,Pizza Place,Coffee Shop,Japanese Restaurant,Café,Pet Store,Bubble Tea Shop


In [93]:
# create final map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ny_merged['Latitude'], ny_merged['Longitude'], ny_merged['Neighborhood'], ny_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [94]:
## Examine Clusters

## Cluster 1
ny_merged.loc[ny_merged['Cluster Labels'] == 0, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,0,Golf Course,Fast Food Restaurant,Pool,Mediterranean Restaurant,Dog Run,Frozen Yogurt Shop,Fried Chicken Joint,Coffee Shop,Comfort Food Restaurant,Gift Shop
1,North York,0,Clothing Store,Fast Food Restaurant,Coffee Shop,Tea Room,Men's Store,Restaurant,Bakery,Women's Store,Toy / Game Store,Deli / Bodega
2,North York,0,Chinese Restaurant,Café,Bank,Japanese Restaurant,Dog Run,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega,Department Store
5,North York,0,Restaurant,Ramen Restaurant,Fast Food Restaurant,Sandwich Place,Pizza Place,Coffee Shop,Japanese Restaurant,Café,Pet Store,Bubble Tea Shop
6,North York,0,Electronics Store,Bank,Park,General Entertainment,Discount Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega
7,North York,0,Pharmacy,Pizza Place,Coffee Shop,Butcher,Discount Store,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega,Department Store
8,North York,0,Park,Food & Drink Shop,Fast Food Restaurant,Women's Store,Discount Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega
10,North York,0,Gym,Asian Restaurant,Coffee Shop,Beer Store,Grocery Store,Bike Shop,Fast Food Restaurant,Italian Restaurant,Japanese Restaurant,Dim Sum Restaurant
11,North York,0,Coffee Shop,Gift Shop,Bank,Pharmacy,Pizza Place,Deli / Bodega,Bridal Shop,Restaurant,Diner,Shopping Mall
12,North York,0,Caribbean Restaurant,Miscellaneous Shop,Metro Station,Bar,Massage Studio,Coffee Shop,Women's Store,Diner,Construction & Landscaping,Cosmetics Shop


In [95]:
## Cluster 2
ny_merged.loc[ny_merged['Cluster Labels'] == 1, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,North York,1,Gym / Fitness Center,Caribbean Restaurant,Café,Basketball Court,Japanese Restaurant,Women's Store,Dog Run,Construction & Landscaping,Cosmetics Shop,Deli / Bodega


In [96]:
## Cluster 3
ny_merged.loc[ny_merged['Cluster Labels'] == 2, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,North York,2,Park,Women's Store,Greek Restaurant,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop
18,North York,2,Fast Food Restaurant,Italian Restaurant,Coffee Shop,Grocery Store,Sandwich Place,Greek Restaurant,Hardware Store,Indian Restaurant,Juice Bar,Liquor Store
19,North York,2,Furniture / Home Store,Accessories Store,Event Space,Miscellaneous Shop,Boutique,Shoe Store,Clothing Store,Coffee Shop,Vietnamese Restaurant,Athletics & Sports
23,North York,2,Baseball Field,Women's Store,Electronics Store,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant


Most of the neighborhoods fall into the first cluster.